## Quantum Counterfeit Coin Problem

## Problem Description

The Counterfeit Coin Problem is a classical puzzle. It has a long history of challenging one’s reasoning power and ingenuity. The problem goes like this: A man has 8 nickels among which there may be a counterfeit coin, which can only be told apart by its weight being different from the others. How can one tell in not more than three weighing whether there is a counterfeit nickel, if so which one it is and whether it is heavier or lighter than a genuine nickel? The balance we are allowed to use only gives the information whether two masses have the same weight or if not which one is heavier of lighter.


### Preparing the environment
First, we prepare the environment. 

In [12]:
# importing all the helpful libraries 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi, cos, acos, sqrt
from qiskit import Aer, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.tools.visualization import plot_histogram

### Setting up the number of coins

Next, we set the number of coins. This determines the quantum superpositions used by the algorithm.

In [2]:
M = 16                   # Maximum number of qubits available
totalCoins = 8        # This number should be up to M-1

if totalCoins < 4 or totalCoins >= M:
    raise Exception("Please use totalCoins between 4 and ", M-1)

### Setting up the index of the false coin

Next, we set the index of the counterfeit coin. This determines the quantum beam balance.

In [3]:
counterfeitCoinIndex = 5     # This should be 0, 1, ..., totalCoins - 1, where we use Python indexing 

if counterfeitCoinIndex < 0 or counterfeitCoinIndex >= totalCoins:
    raise Exception("counterfeitCoinIndex must be between 0 and ", totalCoins-1)

# Querying the quantum beam balance

As in a classical algorithm to find the counterfeit coin, we will use the balance by placing the same number of coins on the left and right pans of the beam. The difference is that in a quantum algorithm, we can query the beam balance in superposition. To query the quantum beam balance, we use a binary query string to encode coins placed on the pans; namely, the binary string `01101010` means to place coins whose indices are 1, 2, 4, and 6 on the pans, while the binary string `01110111` means to place all coins but those with indices 0 and 4 on the pans. Notice that we do not care how the selected coins are placed on the left and right pans, because their results are the same: it is balanced when no false coin is included, and tilted otherwise. 

In our example, because the number of coins is $8$ and the index of false coin is $5$, the query `01101010` will result in balanced (or, $0$), while the query `01110111` will result in tilted (or, $1$). Using two quantum registers to query the quantum balance, where the first register is for the query string and the second register for the result of the quantum balance, we can write the query to the quantum balance (omitting the normalization of the amplitudes): 

\begin{eqnarray}
|01101010\rangle\Big( |0\rangle - |1\rangle \Big) &\xrightarrow{\mbox{Quantum Beam Balance}}& |01101010\rangle\Big( |0\oplus 0\rangle - |1 \oplus 0\rangle \Big) = |01101010\rangle\Big( |0\rangle - |1\rangle \Big)\\
|01110111\rangle\Big( |0\rangle - |1\rangle \Big) &\xrightarrow{\mbox{Quantum Beam Balance}}& |01110111\rangle\Big( |0 \oplus 1\rangle - |1 \oplus 1\rangle \Big) = (-1) |01110111\rangle\Big( |0 \rangle - |1 \rangle \Big)
\end{eqnarray}

Notice that in the above equation, the phase is flipped if and only if the binary query string is $1$ at the index of the false coin. Let $x \in \left\{0,1\right\}^N$ be the $N$-bit query string (that contains even number of $1$s), and let $e_k \in \left\{0,1\right\}^N$ be the binary string which is $1$ at the index of the false coin and $0$ otherwise. Clearly, 

$$
|x\rangle\Big(|0\rangle - |1\rangle \Big) \xrightarrow{\mbox{Quantum Beam Balance}} \left(-1\right) ^{x\cdot e_k} |x\rangle\Big(|0\rangle - |1\rangle \Big), 
$$
where $x\cdot e_k$ denotes the inner product of $x$ and $e_k$. 

Here, we will prepare the superposition of all binary query strings with even number of $1$s. Namely, we want a circuit that produces the following transformation:

$$
|0\rangle \rightarrow \frac{1}{2^{(N-1)/2}}\sum_{x\in \left\{0,1\right\}^N~\mbox{and}~|x|\equiv 0 \mod 2} |x\rangle,
$$

where $|x|$ denotes the Hamming weight of $x$.

To obtain such superposition of states of even number of $1$s, we can perform Hadamard transformation on $|0\rangle$ to obtain superposition of $\sum_{x\in\left\{0,1\right\}^N} |x\rangle$, and check if the Hamming weight of $x$ is even. It can be shown that the Hamming weight of $x$ is even if and only if $x_1 \oplus x_2 \oplus \ldots \oplus x_N = 0$. Thus, we can transform:

\begin{equation}
|0\rangle|0\rangle \xrightarrow{H^{\oplus N}} \frac{1}{2^{N/2}}\sum_x |x\rangle |0\rangle \xrightarrow{\mbox{XOR}(x)} \frac{1}{2^{N/2}}\sum_x |x\rangle |0\oplus x_1 \oplus x_2 \oplus \ldots \oplus x_N\rangle 
\end{equation}

The right-hand side of the equation can be divided based on the result of the $\mbox{XOR}(x) = x_1 \oplus \ldots \oplus x_N$, namely, 

$$
\frac{1}{2^{(N-1)/2}}\sum_{x\in \left\{0,1\right\}^N~\mbox{and}~|x|\equiv 0 \mod 2} |x\rangle|0\rangle + \frac{1}{2^{(N-1)/2}}\sum_{x\in \left\{0,1\right\}^N~\mbox{and}~|x|\equiv 1 \mod 2} |x\rangle|1\rangle.
$$

Thus, if we measure the second register and observe $|0\rangle$, the first register is the superposition of all binary query strings we want. If we fail (observe $|1\rangle$), we repeat the above procedure until we observe $|0\rangle$. Each repetition is guaranteed to succeed with probability exactly half. Hence, after several repetitions we should be able to obtain the desired superposition state. 

In [23]:
# Creating registers
# totalCoins qubits for the binary query string and 1 qubit for working and recording the result of quantum balance
qr = QuantumRegister(totalCoins+1)
# for recording the measurement on qr
cr = ClassicalRegister(totalCoins+1)

circuitName = "Counterfeit Coin Problem"
counterfeitCoinCircuit = QuantumCircuit(qr, cr)
N = totalCoins
# Create uniform superposition of all strings of length N
for i in range(N):
    counterfeitCoinCircuit.h(qr[i])

# Perform XOR(x) by applying CNOT gates sequentially from qr[0] to qr[N-1] and storing the result to qr[N]
for i in range(N):
    counterfeitCoinCircuit.cx(qr[i], qr[N])

# Measure qr[N] and store the result to cr[N]. We continue if cr[N] is zero, or repeat otherwise
counterfeitCoinCircuit.measure(qr[N], cr[N])

# we proceed to query the quantum beam balance if the value of cr[0]...cr[N] is all zero
# by preparing the Hadamard state of |1>, i.e., |0> - |1> at qr[N]
counterfeitCoinCircuit.x(qr[N]).c_if(cr, 0)
counterfeitCoinCircuit.h(qr[N]).c_if(cr, 0)

# we rewind the computation when cr[N] is not zero
for i in range(N):
    counterfeitCoinCircuit.h(qr[i]).c_if(cr, 2**N)


### Constructing the quantum beam balance

The quantum beam balance returns $1$ when the binary query string contains the position of the false coin and $0$ otherwise, provided that the Hamming weight of the binary query string is even. Notice that previously, we constructed the superposition of all binary query strings whose Hamming weights are even. Let $k$ be the position of the false coin, then with regards to the binary query string $|x_1,x_2,\ldots,x_N\rangle|0\rangle$, the quantum beam balance simply returns $|x_1,x_2,\ldots,x_N\rangle|0\oplus x_k\rangle$, that can be realized by a CNOT gate with $x_k$ as control and the second register as target. Namely, the quantum beam balance realizes

$$
|x_1,x_2,\ldots,x_N\rangle\Big(|0\rangle - |1\rangle\Big) \xrightarrow{\mbox{Quantum Beam Balance}} |x_1,x_2,\ldots,x_N\rangle\Big(|0\oplus x_k\rangle - |1 \oplus x_k\rangle\Big) = \left(-1\right)^{x\cdot e_k} |x_1,x_2,\ldots,x_N\rangle\Big(|0\rangle - |1\rangle\Big)
$$

Below we apply the quantum beam balance on the desired superposition state. 

In [24]:
k = counterfeitCoinIndex
# Apply the quantum beam balance on the desired superposition state (marked by cr equal to zero)
counterfeitCoinCircuit.cx(qr[k], qr[N]).c_if(cr, 0)

### References
- https://arxiv.org/pdf/quant-ph/9705041.pdf
- https://arxiv.org/pdf/1009.0416.pdf